In [39]:
import pandas as pd
import numpy as np
import random

In [36]:
dat = pd.read_csv("agesexlkp.csv")
df = dat.drop(columns=['uniqSubNum', 'Notes', 'ResearcherVoice', 'GFTARaw', 'GFTAstatus'])
df.head()

,AgeYrs,CorrectAssignedAge,Sex,ParticipantID
0,4.093087,4,M,cb10
1,5.735797,5,F,PE08
2,3.444216,3,F,SB17
3,4.249144,4,F,am16
4,6.023272,5,F,ay24


In [37]:
np.random.seed(1234)

def assign_id(group):
    group['ppt_group_id'] = np.random.permutation(len(group))
    return group

df = df.groupby(['CorrectAssignedAge', 'Sex'], group_keys=False).apply(assign_id)
df = df.sort_values(by=['CorrectAssignedAge', 'Sex']).reset_index(drop=True)

/var/folders/jb/3gk3p4nn5tlcdvb5zkgy4xh40000gn/T/ipykernel_3809/3514786442.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby(['CorrectAssignedAge', 'Sex'], group_keys=False).apply(assign_id)


In [ ]:
df['set'] = df['ppt_group_id'].apply(lambda x: 'evaluation' if x in [1, 2, 3] else 'training')
df


,AgeYrs,CorrectAssignedAge,Sex,ParticipantID,ppt_group_id,set
0,3.444216,3,F,SB17,7,training
1,3.945243,3,F,CB02,8,training
2,3.192334,3,F,CB11,11,training
3,3.718001,3,F,cb12,2,evaluation
4,3.569037,3,F,CC04,9,training
...,...,...,...,...,...,...
67,5.169062,5,M,mw01,11,training
68,5.256674,5,M,PE04,4,training
69,5.166324,5,M,PE05,10,training
70,5.141684,5,M,pe13,9,training


In [45]:
df['transcriber'] = 'NA'

# Assign 'Both Caroline and Carly' where ppt_age_id == 1
df.loc[df['ppt_group_id'] == 1, 'transcriber'] = 'Both Caroline and Carly'

# Get only rows where set == 'evaluation' and ppt_age_id != 1
eval_mask = (df['set'] == 'evaluation') & (df['ppt_group_id'] != 1)
eval_indices = df[eval_mask].index.tolist()

# Shuffle and split indices equally between Caroline and Carly
random.seed(1234)
random.shuffle(eval_indices)
half = len(eval_indices) // 2
caroline_indices = eval_indices[:half]
carly_indices = eval_indices[half:]

# Assign transcribers
df.loc[caroline_indices, 'transcriber'] = 'Caroline'
df.loc[carly_indices, 'transcriber'] = 'Carly'

df.to_csv("participant_sets.csv", index=False)